In [ ]:
!pip install --user pycocotools

In [14]:
import csv
from shutil import copyfile
from pycocotools.coco import COCO
from tqdm import tqdm
from torchvision import models
import torch.nn as nn

In [2]:
#make directory and get annotations for training and testing
!mkdir data
!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip -P ./data/
!unzip ./data/captions_train-val2014.zip -d ./data/
!rm ./data/captions_train-val2014.zip

--2020-02-18 17:38:47--  http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip
Resolving web.ucsd.edu (web.ucsd.edu)... 132.239.1.230, 132.239.1.231
Connecting to web.ucsd.edu (web.ucsd.edu)|132.239.1.230|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 19673183 (19M) [application/octet-stream Charset=UTF-8]
Saving to: ‘./data/captions_train-val2014.zip’

captions_train-val2 100%[===================>]  18.76M  --.-KB/s    in 0.09s   

2020-02-18 17:38:47 (213 MB/s) - ‘./data/captions_train-val2014.zip’ saved [19673183/19673183]

Archive:  ./data/captions_train-val2014.zip
  inflating: ./data/annotations/captions_train2014.json  
  inflating: ./data/annotations/captions_val2014.json  


In [3]:
!mkdir data/images
!mkdir data/images/train
!mkdir data/images/test

In [4]:
coco = COCO('./data/annotations/captions_train2014.json')

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


In [5]:
#get ids of training images
with open('TrainImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    trainIds = list(reader)
    
trainIds = [int(i) for i in trainIds[0]]

In [6]:
for img_id in trainIds:
    path = coco.loadImgs(img_id)[0]['file_name']
    copyfile('/datasets/COCO-2015/train2014/'+path, './data/images/train/'+path)

In [7]:
cocoTest = COCO('./data/annotations/captions_val2014.json')

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!


In [8]:
with open('TestImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    testIds = list(reader)
    
testIds = [int(i) for i in testIds[0]]

In [11]:
for img_id in testIds:
    path = cocoTest.loadImgs(img_id)[0]['file_name']
    copyfile('/datasets/COCO-2015/val2014/'+path, './data/images/test/'+path)

In [17]:
resnet_model = models.resnet50(pretrained=True)
print(resnet_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
class resnet_encoder(nn.Module):
    def __init__(self, num_classes, fine_tuning=False):
        super(resnet_encoder, self).__init__()
        for param in resnet_model.parameters():
            params.requires_grad = fine_tuning
        self.conv1 = resnet_model.conv1
        self.bn1 = resnet_model.bn1
        self.relu = resnet_model.relu
        self.maxpool = resnet_model.maxpool
        self.layer1 = resnet_model.layer1
        self.layer2 = resnet_model.layer2
        self.layer3 = resnet_model.layer3
        self.layer4 = resnet_model.layer4
        self.avgpool = resnet_model.avgpool
        self.last = nn.Linear(2048, num_classes)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = self.last(out)
        
        return out
        

In [ ]:
class resnet_deocder(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers, output_size):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.input2hidden = nn.Linear(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers)
        self.hidden2tag = nn.Linear(hidden_dim, output_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        h = torch.zeros(self.num_layers,1,self.hidden_dim)
        c = torch.zeros(self.num_layers,1,self.hidden_dim)
        return (Variable(h.cuda()), Variable(c.cuda()))

    def forward(self, x):
        x_len = len(x)
        x = self.input2hidden(x)
        x, self.hidden = self.lstm(x.view(x_len, 1, -1), self.hidden)
        x = self.hidden2tag(x.view(x_len, -1))
        return x